# Nhập môn trí tuệ nhân tạo
## Thông tin
### Nhóm: 16
### Thành viên:
- Đào Nhật Minh - 20200392
- Hoàng Vân Trường - 20205134
- Nguyễn Thu Trang - 20205035
- Trần Anh Tuấn - 20205041
- Nguyễn Giang Nam - 20205103

### Giảng viên hướng dẫn:
- PGS. Trần Đình Khang

## Giới thiệu:

### Mục tiêu

### Mục lục
- [Các thực thể](#entity)
- [Đọc file dữ liệu](#readData)
- [Tìm đường gần nhất từ 1 điểm](#nearestRoad)
- [Hiển thị bản đồ](#displayMap)
- [Tìm đường](#findPath)
- [Ứng dụng](#application)


Chạy các lệnh sau nếu máy tính không có sẵn thư viện

In [ ]:
#python3.10 -m pip install numpy
#python3.10 -m pip install matplotlib 
#python3.10 -m pip install opencv-python
#python3.10 -m pip install ipywidget

In [ ]:
import json
import numpy as np
from queue import PriorityQueue
import codecs
import cv2
import matplotlib.pyplot as plt
import ipywidgets
from tkinter import *
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import copy
import os
from PIL import Image, ImageTk

In [ ]:
class Toolbar(NavigationToolbar2Tk):
    def set_message(self, s):
        pass

<a name = "entity"/>

## 1. Các thực thể

In [ ]:
class Street:
    def __init__(self, id, name):
        self.id = id
        self.name = name
    
class Point:
    def __init__(self, id, x, y, streetList: list):
        self.id = id
        self.x = x
        self.y = y
        self.streetList = streetList
    def getId(self) -> int:
        return self.id
    def getPosition(self) -> list:
        return [self.x, self.y]
    def getStreetList(self) -> list:
        return self.streetList

road_id = 0
class Road:
    def __init__(self , point1: Point, point2: Point, oneWayRoad):
        global road_id
        self.id = road_id
        road_id += 1
        self.point1 = point1
        self.point2 = point2
        self.oneWayRoad = oneWayRoad
    def getPoint1(self)->Point:
        return self.point1
    def getPoint2(self)->Point:
        return self.point2
    def getOneWayRoad(self):
        return self.oneWayRoad
    def getLength(self):
        if self.length != -1:
            x1, y1 = self.getPoint1().getPosition()
            x2, y2 = self.getPoint2().getPosition()
            self.length = abs((x1-x2)**2+(y1-y2)**2)**(1/2)
        return self.length

class Graph:
    next_point_dict = {}
    def __init__(self):
        self.next_point_dict = {}
    def add_road(self, road_to_add: Road):
        p1Id = road_to_add.point1.getId()
        p2Id = road_to_add.point2.getId()
        if p1Id not in self.next_point_dict:
            self.next_point_dict[p1Id] = []
        if p2Id not in self.next_point_dict:
            self.next_point_dict[p2Id] = []
        if road_to_add.oneWayRoad!= 1:
            self.next_point_dict[p2Id] += [p1Id]
        if road_to_add.oneWayRoad!= -1:
            self.next_point_dict[p1Id] += [p2Id]
    def get_next_point_dict(self):
        return self.next_point_dict

<a name = "readData"/>

## 2. Đọc file dữ liệu

In [ ]:
scale_width_ratio = 0
scale_height_ratio = 0
x_max = 105.850260
y_max = 21.044922
x_min = 105.844521
y_min = 21.039959
# Input mode = 0: nhập điểm bắt đầu
# Input mode = 1: Nhập điểm đích
streetDict = {}
pointDict = {}
roadDict = {}
graph = Graph()
g_input_mode = 0 
g_x_st = g_y_st = g_x_t = g_y_t = 0

In [ ]:
def read_data_from_file():
    global graph
    f = codecs.open("data.json", 'r', encoding='utf-8')
    fileContent = f.read()
    dataLoad = json.loads(fileContent)
    
    for street_id in dataLoad["street"]:
        newStreet = Street(street_id, dataLoad["street"][street_id])
        streetDict[street_id] = newStreet
    for point_id in dataLoad["point"]:
        tmpPoint = dataLoad["point"][point_id]
        newPoint = Point(int(point_id), tmpPoint["x"], tmpPoint["y"], tmpPoint["streetList"])
        pointDict[int(point_id)] = newPoint
    
    road_id = 0
    for road in dataLoad["road"]:
        newRoad = Road(pointDict[road["point1"]], pointDict[road["point2"]], road["oneWayRoad"])
        roadDict[road_id] = newRoad
        graph.add_road(newRoad)
        road_id = road_id + 1

def reset_data():
    global streetDict
    global pointDict
    global roadDict
    global graph
    streetDict = {}
    pointDict = {}
    roadDict = {}
    graph = Graph()
    read_data_from_file()

<a name = "nearestRoad"/>

## 3. Tìm đường gần nhất từ 1 điểm

In [ ]:
def getNearestRoad(x, y):
    min_dis = np.inf
    nearest_line_index = -1
    nearest_point = [-1, -1]
    for i in range(len(roadDict)):
        distance , point = calculateDistance(x, y, roadDict[i])
        if distance < min_dis:
            nearest_line_index = i
            nearest_point = point
            min_dis = distance
    return roadDict[nearest_line_index], nearest_point

def calculateDistance(x, y, road: Road):
    x1,y1 = road.getPoint1().getPosition()
    x2,y2 = road.getPoint2().getPosition()
    u = [x2 - x1, y2 - y1]
    v = [x - x1, y - y1]
    proj_v_on_u = [(v[0]*u[0] + v[1]*u[1])/ (u[0]**2 + u[1]**2) * u[0], 
                   (v[0]*u[0] + v[1]*u[1])/ (u[0]**2 + u[1]**2) * u[1]]
    C = [x1 + proj_v_on_u[0], y1 + proj_v_on_u[1]]
    if (C[0] - x1)*(C[0] - x2) <= 0 and (C[1] - y1) * (C[1] - y2) <= 0:
        return np.sqrt((x - C[0])**2 + (y - C[1])**2), C
    else:
        dis_point1 = np.sqrt((x - x1)**2 + (y - y1)**2)
        dis_point2 = np.sqrt((x - x2)**2 + (y - y2)**2)
    if dis_point1 < dis_point2:
        return dis_point1, [x1, y1]
    else:
        return dis_point2, [x2, y2]


<a name = "displayMap"/>

## 4. Hiển thị bản đồ & Event Handle

In [ ]:
def pointTransform(position):
    x_temp, y_temp = position
    return [int((x_temp - x_min) * scale_width_ratio), 
            int((y_max - y_temp) * scale_height_ratio)]
def reversePointTrainform(position):
    x_temp, y_temp = position
    return [x_temp / scale_width_ratio + x_min, 
            y_max - y_temp / scale_height_ratio]

In [ ]:
def onclick(event):
    global g_input_mode
    global g_x_st
    global g_y_st
    global g_x_t
    global g_y_t
    global clone_map_fig
    if g_input_mode == 0:
        g_x_st, g_y_st = reversePointTrainform([event.xdata, event.ydata])
        st_x_label["text"], st_y_label["text"] = [format(z, '.6f') for z in [g_x_st, g_y_st]]
        drawed_map_fig.get_axes()[0].plot(event.xdata, event.ydata, marker="o", markersize=5, 
                                          markeredgecolor="black", markerfacecolor="green")
        g_input_mode = 1
    elif g_input_mode == 1:
        g_x_t, g_y_t = reversePointTrainform([event.xdata, event.ydata])
        t_x_label["text"], t_y_label["text"] = [format(z, '.6f') for z in [g_x_t, g_y_t]]
        drawed_map_fig.get_axes()[0].plot(event.xdata, event.ydata, marker="o", markersize=5, 
                                          markeredgecolor="black", markerfacecolor="blue")
        tmpSt, tmpT = pointTransform([g_x_st, g_y_st]), pointTransform([g_x_t, g_y_t])
        drawed_map_fig.get_axes()[0].plot([tmpSt[0], tmpT[0]],[tmpSt[1], tmpT[1]], 'r--')
        g_input_mode = -1
    scatter.draw()
def resetMode():
    global g_input_mode
    g_input_mode = 0
    global drawed_map_fig
    global toolbarFrame
    global scatter
    drawed_map_fig = read_image()
    for widget in image_frame.winfo_children():
        widget.destroy()
    for widget in toolbarFrame.winfo_children():
        widget.destroy()    
    scatter = FigureCanvasTkAgg(drawed_map_fig, image_frame)
    scatter.draw()
    scatter.get_tk_widget().pack(fill='both', expand=True)
    toolbarFrame = Frame(left_frame, width=180)
    toolbarFrame.grid(row=2, column=0, padx = 5, pady = 5, sticky="ns")
    toolbar = Toolbar(scatter, toolbarFrame)
    
def read_image():
    fig, ax = plt.subplots(figsize=(8, 6))
    global scale_width_ratio
    global scale_height_ratio
    global img
    img = cv2.imread("./map.png")
    img_height, img_width, color_channel = img.shape
    ax.imshow(img)
    scale_width_ratio = img_width/(x_max-x_min)
    scale_height_ratio = img_height/(y_max-y_min)
    ax.set_xlim(0, img_width)
    ax.set_ylim(0, img_height)
    ax.invert_yaxis()

    for pnt in pointDict:
        x_temp, y_temp = pointDict[pnt].getPosition()
        x_temp, y_temp = pointTransform([x_temp, y_temp])
        nextP = graph.get_next_point_dict()
        for pointId in nextP:
            p1x, p1y = pointTransform(pointDict[pointId].getPosition())
            point2 = nextP[pointId]
            for pt2Id in point2:
                p2x, p2y = pointTransform(pointDict[pt2Id].getPosition())
                ax.plot([p1x, p2x], [p1y, p2y], color = "#0e34b0")  
    for pnt in pointDict:
        x_temp, y_temp = pointTransform(pointDict[pnt].getPosition())
        ax.add_patch(plt.Circle((x_temp,y_temp),10, color='r'))
    fig.canvas.mpl_connect('button_press_event', onclick)
    return fig


<a name = "findPath"/>

## 5. Tìm đường

In [ ]:
def calculateLength(point1: Point, point2: Point):
    x1, y1 = point1.getPosition()
    x2, y2 = point2.getPosition()
    return ((x1-x2)**2+(y1-y2)**2)**(1/2)

In [ ]:
# KQ return của hàm findPath là 1 list gồm các step, trong đó
#   + Loại 1: Step tính đường di chuyển (list): [point1's id, point2's id, weight]
#   + Loại 2: Cố định kết quả của point (number): point 
def findPath(x_start, y_start, x_target, y_target): 
    print("target: ", x_target, y_target)
    reset_data()
    trace = {}
    nearestRoad_st, nearestPoint_st = getNearestRoad(x_start, y_start)
    nearestRoad_t, nearestPoint_t = getNearestRoad(x_target, y_target)
    nearestTargetPointId = 0
    nearestStPointId = -1
    startPointId = -2
    targetPointId = -3
    outputResult = []
    print("target: ", x_target, y_target)
    visitedPoint = [False]*(len(pointDict)+5)
    nextPointDict = graph.get_next_point_dict()
    # create 4 point
    pointDict[nearestTargetPointId] = Point(nearestTargetPointId, nearestPoint_t[0], nearestPoint_t[1], [])
    pointDict[nearestStPointId] = Point(nearestStPointId, nearestPoint_st[0], nearestPoint_st[1], [])
    pointDict[startPointId] = Point(startPointId, x_start, y_start, [])
    print("target: ", x_target, y_target)
    pointDict[targetPointId] = Point(targetPointId, x_target, y_target, [])
    # add 2 road to nearest target point 
    if nearestRoad_t.getOneWayRoad() != -1:
        nextPointDict[nearestRoad_t.getPoint1().getId()].append(nearestTargetPointId);
    if nearestRoad_t.getOneWayRoad() != 1:
        nextPointDict[nearestRoad_t.getPoint2().getId()].append(nearestTargetPointId);
    
    pathPriorityQueue = PriorityQueue()
    w1 = calculateLength(nearestRoad_st.getPoint1(), pointDict[nearestStPointId])
    pathPriorityQueue.put((w1, nearestRoad_st.getPoint1().getId(), nearestStPointId))
    w2 = calculateLength(nearestRoad_st.getPoint2(), pointDict[nearestStPointId])
    pathPriorityQueue.put((w2, nearestRoad_st.getPoint2().getId(), nearestStPointId))
    
    outputResult.append([startPointId, nearestStPointId, 0])
    outputResult.append([nearestStPointId, nearestRoad_st.getPoint1().getId(), w1])
    outputResult.append([nearestStPointId, nearestRoad_st.getPoint2().getId(), w2])
    
    while not pathPriorityQueue.empty():
        tempLength, tempPointId, rootId = pathPriorityQueue.get()
        if visitedPoint[tempPointId] == True:
            continue
        trace[tempPointId] = rootId
        outputResult.append(tempPointId)
        if tempPointId == nearestTargetPointId: 
            break
        visitedPoint[tempPointId] = True
        for nextPointId in nextPointDict[tempPointId]:         
            if visitedPoint[nextPointId] == False:
                tmp_weight = tempLength + calculateLength(pointDict[tempPointId], pointDict[nextPointId])
                outputResult.append([tempPointId, nextPointId, tmp_weight])
                pathPriorityQueue.put((tmp_weight, nextPointId, tempPointId))
    outputResult.append([nearestTargetPointId, targetPointId, 0])
    outputResult.append(targetPointId)
    return outputResult


<a name="application">
    
## 6. Ứng dụng

In [ ]:
def find_all_road():
    images_folder_path = os.path.join(os.getcwd(),'output_images')
    file_list = os.listdir(images_folder_path)
    for file_name in file_list:
        file_path = os.path.join(images_folder_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
    global steps
    for step in range(0, len(steps)):
        print(step)
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.imshow(img)
        marked = {}
        for key in pointDict.keys():
            marked[key] = True
        for i in range(step -1, -1, -1):
            if type(steps[i]).__name__ == 'list':
                p1Id, p2Id, weight = steps[i]
                p1 = pointTransform(pointDict[p1Id].getPosition())
                p2 = pointTransform(pointDict[p2Id].getPosition())
                print(p1Id, p1, p2Id, p2)
                color_type = 'g--' if marked[p2Id] else 'b-'
                plt.plot([p1[0], p2[0]], [p1[1], p2[1]], color_type)
            else:
                marked[steps[i]] = False
        fig.savefig(os.path.join(os.getcwd(),'output_images','{:03d}.png'.format(step)))
        plt.close(fig)

In [ ]:
def show_image(image_index):
    image_label.config(image=image_list[image_index])
    image_label.image = image_list[image_index]
def show_next_image():
    global image_index 
    image_index += 1
    if image_index >= len(image_list):
        image_index = 0
    show_image(image_index)

def show_previous_image():
    global image_index 
    image_index -= 1
    if image_index < 0:
        image_index = len(image_list) - 1
    show_image(image_index)

def calculate_ways():
    global steps
    steps = findPath(g_x_st, g_y_st, g_x_t, g_y_t)
    find_all_road()
    new_window = tk.Toplevel(root)
    new_window.title('New Window')

    global image_list
    image_list = []
    images_folder_path = os.path.join(os.getcwd(),'output_images')
    file_list = sorted(os.listdir(images_folder_path))
    for file_name in file_list:
        file_path = os.path.join(images_folder_path, file_name)
        if os.path.isfile(file_path):
            image_tk = ImageTk.PhotoImage(Image.open(file_path))
            image_list.append(image_tk)
    
    # Set the initial image index
    global image_index 
    image_index = 0
    # Create the image label
    global image_label
    image_label = tk.Label(new_window)
    image_label.grid(row=0, column=0, columnspan=2)
    
    # Show the initial image
    show_image(image_index)
    
    # Create the back and next buttons
    back_btn = tk.Button(new_window, text="← Back", command=show_previous_image)
    next_btn = tk.Button(new_window, text="Next →", command=show_next_image)
    back_btn.grid(row=1, column=0)
    next_btn.grid(row=1, column=1)


In [ ]:
root = Tk()
root.title('Demo Application')
root.maxsize(1280, 720)
g_input_mode = 0
# left frame
left_frame = Frame(root, width=200, height=400)
left_frame.grid(row=0, column=0, padx=10, pady=5)

# Create 2 menu within left_frame
l_up_menu = Frame(left_frame, width=180, height=185, highlightbackground="black", highlightcolor="black", highlightthickness=1)
l_up_menu.grid(row=0, column=0, padx=5, pady=5)
l_down_menu = Frame(left_frame, width=180, height=185, highlightbackground="black", highlightcolor="black", highlightthickness=1)
l_down_menu.grid(row=1, column=0, padx=5, pady=5)

## First menu for position
stPositionLabel = Label(l_up_menu, text="Start Position")
stPositionLabel.grid(row=0, column=0, columnspan=2)
xLabel = Label(l_up_menu, text="x = ")
xLabel.grid(row=1, column=0)
st_x_label = Label(l_up_menu, text="")
st_x_label.grid(row=1, column=1)
yLabel = Label(l_up_menu, text="y = ")
yLabel.grid(row=2, column = 0)
st_y_label = Label(l_up_menu, text="")
st_y_label.grid(row=2, column=1)

tPositionLabel = Label(l_up_menu, text="Target Position")
tPositionLabel.grid(row=3, column=0, columnspan=2)
xLabel = Label(l_up_menu, text="x = ")
xLabel.grid(row=4, column=0)
t_x_label = Label(l_up_menu, text="")
t_x_label.grid(row=4, column=1)
yLabel = Label(l_up_menu, text="y = ")
yLabel.grid(row=5, column = 0)
t_y_label = Label(l_up_menu, text="")
t_y_label.grid(row=5, column=1)
## Second menu for button
resetButton = Button(l_down_menu, text = "RESET", command = resetMode, padx=5, pady=5)
resetButton.grid(row = 0, column = 0)
startBuutton = Button(l_down_menu, text = "START", command = calculate_ways, padx=5, pady=5)
startBuutton.grid(row = 1, column = 0)

# right frame
right_frame = Frame(root, width=650, height=650, bg='grey')
right_frame.grid(row=0, column=1, padx=10, pady=5)
# Image
image_frame = Frame(right_frame, width = 640, height = 640)
image_frame.grid(row=0, column=0, padx = 5, pady = 5)
reset_data()
drawed_map_fig = read_image()
scatter = FigureCanvasTkAgg(drawed_map_fig, image_frame)
scatter.draw()
scatter.get_tk_widget().pack(fill='both', expand=True)

## toolbar on Leftframe
toolbarFrame = Frame(left_frame, width=180)
toolbarFrame.grid(row=2, column=0, padx = 5, pady = 5, sticky="ns")
toolbar = Toolbar(scatter, toolbarFrame)

root.mainloop()